In [4]:
import pandas as pd
import numpy as np
import json
from pathlib import Path

In [5]:
class FeatureExtractor:
    def __init__(self, window_size_min: int = 10):
        self.window_size_min = window_size_min

    def load_raw_data(self, filepath: str) -> pd.DataFrame:
        path = Path(filepath)
        if path.suffix == '.csv':
            df = pd.read_csv(filepath, parse_dates=['timestamp'])
        elif path.suffix == '.json':
            with open(filepath) as f:
                data = json.load(f)
            df = pd.DataFrame(data)
            df['timestamp'] = pd.to_datetime(df['timestamp'])
        else:
            raise ValueError(f"Unsupported format: {path.suffix}")

        required = {'timestamp', 'asset_id', 'vibration_rms', 'temperature_c', 'load_percent'}
        assert required.issubset(set(df.columns)), f"Missing columns: {required - set(df.columns)}"
        print(f"✓ Loaded {len(df):,} raw samples from {filepath}")
        return df

    def map_component_type(self, asset_id: str) -> str:
        asset_id_lower = asset_id.lower()
        if 'pump' in asset_id_lower:
            return 'PUMP'
        elif 'conveyor' in asset_id_lower:
            return 'CONVEYOR'
        elif 'compressor' in asset_id_lower:
            return 'COMPRESSOR'
        else:
            return 'UNKNOWN'

    def extract_features(self, df: pd.DataFrame) -> list:
        features_list = []
        window = self.window_size_min

        for asset_id, asset_df in df.groupby('asset_id'):
            asset_df = asset_df.sort_values('timestamp').reset_index(drop=True)
            component_type = self.map_component_type(asset_id)

            for i in range(len(asset_df)):
                row = asset_df.iloc[i]

                # Base measurements
                vibration_rms = float(row['vibration_rms'])
                temperature_c = float(row['temperature_c'])
                load_avg = float(row['load_percent'])  # Context only — no trend/delta

                # Vibration trend (degradation rate) — PRIMARY diagnostic
                if i >= window - 1:
                    x = np.arange(window)
                    y_vib = asset_df['vibration_rms'].values[i - window + 1:i + 1]
                    vibration_trend = float(np.polyfit(x, y_vib, 1)[0])
                    vibration_delta = float(row['vibration_rms'] - asset_df['vibration_rms'].iloc[i - window + 1])
                else:
                    vibration_trend = 0.0
                    vibration_delta = 0.0

                # Temperature delta (thermal lag evidence) — SECONDARY confirmation
                if i >= window - 1:
                    temperature_delta = float(row['temperature_c'] - asset_df['temperature_c'].iloc[i - window + 1])
                else:
                    temperature_delta = 0.0

                # CRITICAL DESIGN CHOICE: NO load_trend / load_delta
                # Reason: Load is an external driver (production demand), not a degradation indicator.
                # Including load trends would confuse root cause analysis in XAI layer.

                feature_packet = {
                    "timestamp": row['timestamp'].strftime("%Y-%m-%dT%H:%M:%SZ"),
                    "component": component_type,
                    "features": {
                        "vibration_rms": round(vibration_rms, 2),       # PRIMARY
                        "vibration_trend": round(vibration_trend, 3),   # Degradation rate
                        "vibration_delta": round(vibration_delta, 2),   # Early warning
                        "temperature_c": round(temperature_c, 1),       # SECONDARY
                        "temperature_delta": round(temperature_delta, 1), # Thermal lag proof
                        "load_avg": round(load_avg, 1)                  # CONTEXT only
                    }
                }
                features_list.append(feature_packet)

        print(f"✓ Extracted {len(features_list):,} feature packets (6 high-value features)")
        return features_list

    def export_features(self, features: list, output_path: str = "extracted_features.json"):
        with open(output_path, 'w') as f:
            json.dump(features, f, indent=2)
        print(f"✅ Features exported: {output_path}")
        print(f"   Example packet:\n{json.dumps(features[0], indent=2)}")

In [6]:
def main():
    RAW_DATA_PATH = "simulated_assets.csv"
    OUTPUT_PATH = "extracted_features.json"

    extractor = FeatureExtractor(window_size_min=10)
    raw_df = extractor.load_raw_data(RAW_DATA_PATH)
    features = extractor.extract_features(raw_df)
    extractor.export_features(features, OUTPUT_PATH)

    # Validation
    assert len(features) > 0
    assert all(k in features[0]['features'] for k in [
        'vibration_rms', 'vibration_trend', 'vibration_delta',
        'temperature_c', 'temperature_delta', 'load_avg'
    ]), "Missing required features"
    print("\n✅ FEATURE EXTRACTION COMPLETE — 6 high-value features ready for XAI layer")


if __name__ == "__main__":
    main()

✓ Loaded 30,240 raw samples from simulated_assets.csv
✓ Extracted 30,240 feature packets (6 high-value features)
✅ Features exported: extracted_features.json
   Example packet:
{
  "timestamp": "2026-01-01T00:00:05Z",
  "component": "COMPRESSOR",
  "features": {
    "vibration_rms": 2.78,
    "vibration_trend": 0.0,
    "vibration_delta": 0.0,
    "temperature_c": 44.5,
    "temperature_delta": 0.0,
    "load_avg": 73.2
  }
}

✅ FEATURE EXTRACTION COMPLETE — 6 high-value features ready for XAI layer
